## Tutorial: Motion Primitives Generator

This tutorial demonstrates how are the motion primitives generated. You can find all related files in the [CommonRoad Search](https://gitlab.lrz.de/tum-cps/commonroad-search) repository, under folder GSMP/tools/motion_primitive_generator/.

The main components are:
* **motion_primitives_config.yaml** in which the configuration is set;
* **implementation.py** in which the main functions are implemented; and
* the script below to execute the functions.

Before proceeding, you should make sure the configuration file is set correctly, which consists of the following parameters:
* **output setting**:
    * output_directory: output directory for generated motion primitives.
* **vehicle setting**:
    * veh_type_id: id of vehicle type. 1: FORD_ESCORT, 2: BMW_320i, 3: VW_VANAGON
* **primitive setting**:
    * T: time length of trajectory [s].
    * dt_simulation: time step for forwad state simulation [s].
    * velocity_sample_min: minimum sampling velocity [m/s].
    * velocity_sample_max: maximum sampling velocity [m/s].
    * num_sample_velocity: number of velocity samples.
    * steering_angle_sample_min: minimum sampling angle [rad]. Note that here we only consider steering to one side, as we will mirror the primitives afterwards.
    * steering_angle_sample_max: maximum sampling angle [rad]. If set to 0, it will be assigned the maximum permissible value given by the selected vehicle parameter.
    * num_sample_steering_angle: number of steering angle samples
* **sample trajectory setting**: 
    * num_segment_trajectory: number of segments in sample trajectories
    * num_simulations: number of sample trajectories to be generated

Note: the paths can be either **relative** to this script, or **absolute**.

### Load configuration file and parameters

In [ ]:
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt

# import helper functions
from implementation import Parameter, load_configuration, generate_motion_primitives, create_mirrored_primitives, \
save_motion_primitives, compute_number_successors_average, compute_number_successors_average, \
generate_sample_trajectories

# specify path to configuration file
path_file_config = "../configuration/motion_primitives_config.yaml"

# load configuration file
configuration = load_configuration(path_file_config)

# parse configuration and generate parameter object
parameters = Parameter(configuration)

### Generate motion primitives
The attributes of each of the states in a motion primitive are: x position, y position, steering angle, velocity in x direction, orientation and time step

In [ ]:
list_traj_accepted = generate_motion_primitives(parameters)

### Plot generated motion primitives

In [ ]:
for traj in list_traj_accepted:
    list_x = [state.position[0] for state in traj.state_list]
    list_y = [state.position[1] for state in traj.state_list]
    plt.plot(list_x, list_y)
    
plt.axis('equal')
plt.show()

### Create mirrored primitives
As we only computed primitives that have a positive (negative) steering angle, we mirror them to get the other half of primitives.

In [ ]:
list_traj_accepted_mirrored = create_mirrored_primitives(list_traj_accepted, parameters)
print("Total number of primitives (mirrored included): ", len(list_traj_accepted_mirrored))

### Check average number of successors
We can inspect how many successors does every primitive have on average.

In [ ]:
num_average_successors = compute_number_successors_average(list_traj_accepted_mirrored)
print("average number of successors: ", num_average_successors)

### Plot final motion primitives
We can plot the final generated motion primitives.

In [ ]:
# fig = plt.figure()
for traj in list_traj_accepted_mirrored:
    list_x = [state.position[0] for state in traj.state_list]
    list_y = [state.position[1] for state in traj.state_list]
    plt.plot(list_x, list_y)
    
plt.axis('equal')
plt.show()

### Generate sample trajectories
We can generate sample trajectories with the motion primitives that we have just generated

In [ ]:
generate_sample_trajectories(list_traj_accepted_mirrored, parameters)

### Save the motion primitives into xml file
The xml containing generated motion primitives are output to the directory specified in the configuration

In [ ]:
save_motion_primitives(list_traj_accepted_mirrored, parameters)